In [2]:
import onnx
from onnx import helper,TensorProto
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.core.datatype import DataType
import numpy as np
from qonnx.util.basic import gen_finn_dt_tensor
import onnx.version_converter as vc
from onnx.backend.test.case.node import expect

from qonnx.transformation.infer_shapes import InferShapes

model = ModelWrapper("tinyyolo-20210831.onnx")
model.get_initializer("737")


/workspace/qonnx/src/qonnx/core/modelwrapper.py:93: UserWarning: Some old-style domain attributes were automatically converted to new-style,
                i.e. domain=finn to domain=qonnx.custom_op.<general|fpgadataflow|...>
  warnings.warn(


array([0])

### Convolutional Node

In [17]:
from qonnx.custom_op.general.im2col import compute_conv_output_dim

kernel_size, stride, pad = 1,1,0

depthwise = False
in_feature_dim = 16
in_chn = 128

idt = DataType["UINT8"]

group = 1
out_chn = 64
conv_param_shape = [out_chn, in_chn, kernel_size, kernel_size]

total_pad = 2 * pad
out_feature_dim = compute_conv_output_dim(
    in_feature_dim, kernel_size, stride, total_pad
)

input_shape = [1, in_chn, in_feature_dim, in_feature_dim]
output_shape = [1, out_chn, out_feature_dim, out_feature_dim]

conv_weight_dt = DataType["INT2"]

conv_config = {}
conv_config["dilations"] = [1, 1]
conv_config["group"] = group
conv_config["kernel_shape"] = [kernel_size, kernel_size]
conv_config["pads"] = [pad, pad, pad, pad]
conv_config["strides"] = [stride, stride]

top_in = helper.make_tensor_value_info("top_in", TensorProto.FLOAT, input_shape)
conv_out = helper.make_tensor_value_info("conv_out", TensorProto.FLOAT, output_shape)
value_info_conv = [
    helper.make_tensor_value_info("c1", TensorProto.FLOAT, conv_param_shape)
]
conv_node = helper.make_node("Conv", ["top_in", "c1"], ["conv_out"], **conv_config)
modelproto = helper.make_model(
    helper.make_graph(
        name="conv_test",
        inputs=[top_in],
        outputs=[conv_out],
        value_info=value_info,
        nodes=[conv_node],
    )
)

model = ModelWrapper(modelproto)
model.set_tensor_datatype("c1", conv_weight_dt)
model.set_initializer("c1", gen_finn_dt_tensor(conv_weight_dt, conv_param_shape))

model.save("onnx_model/conv_model.onnx")

In [13]:
# shape node 
shape_in = helper.make_tensor_value_info("conv_out", TensorProto.FLOAT, [1,64,16,16])
shape_out = helper.make_tensor_value_info("735", TensorProto.INT64,[4])
shape_node = helper.make_node(
            "Shape",
            inputs = ["conv_out"],
            outputs = ["735"])
shape_graph = helper.make_graph(
                [shape_node],
                inputs = [shape_in],
                outputs = [shape_out],
                name = "shape_graph"
                )
shape_model = ModelWrapper(helper.make_model(shape_graph))
shape_model.transform(InferShapes())
shape_model.save("onnx_model/Shape_model.onnx")

### Slice node


In [14]:
input_shape = [4]
output_shape = [2]
param_shape = [1]
idt = DataType["UINT8"]
param_dt = DataType["INT64"]
slice_in = helper.make_tensor_value_info("735", TensorProto.INT64, input_shape)
slice_out = helper.make_tensor_value_info("slice_out", TensorProto.INT64, output_shape)
slice_attr = {}
slice_attr["starts"] = np.array([0],dtype=np.int64)
slice_attr["ends"] = np.array([2],dtype=np.int64)
slice_attr["axes"] = np.array([0],dtype=np.int64)

# value_info = [
#         helper.make_tensor_value_info("starts", TensorProto.INT8, param_shape),
#         helper.make_tensor_value_info("ends",   TensorProto.INT8, param_shape),
#         helper.make_tensor_value_info("axes",   TensorProto.INT8, param_shape),
# ]



slice_node = helper.make_node(
            "Slice",
            inputs  = ["735"],#,"starts","ends","axes"],
            outputs = ["slice_out"],
            **slice_attr
            )

slice_graph = helper.make_graph(
                [slice_node],
                inputs = [slice_in],
                outputs = [slice_out],
                name = "slice_graph"
                )

model_config = {}
model_config["opset_imports"] = [helper.make_operatorsetid("",9)]

slice_model = ModelWrapper(helper.make_model(slice_graph,**model_config))
slice_model.transform(InferShapes())
onnx.checker.check_model(slice_model.model)
slice_model.save("onnx_model/Slice_model.onnx")

### Concat node

In [15]:
concat_in = helper.make_tensor_value_info("slice_out", TensorProto.INT64, [2])
p1 = helper.make_tensor_value_info("839", TensorProto.INT64,[2])
top_out = helper.make_tensor_value_info("top_out", TensorProto.INT64,[4])
concat_config = {}
concat_config["axis"] = np.int64(0)
concat_node = helper.make_node(
            "Concat",
            inputs = ["slice_out","839"],
            outputs = ["top_out"],
            **concat_config)

concat_model = helper.make_model(
    
    helper.make_graph(    
        [concat_node],
        inputs=[concat_in],
        outputs = [top_out],
        name = "concat_graph",
        value_info=[p1])
)

Concat_model = ModelWrapper(concat_model)
Concat_model.set_initializer("839",np.array([10,10]))
Concat_model.transform(InferShapes())
Concat_model.save("Concat_model.onnx")

### Create Full graph

In [28]:
# Create graph
value_info = [
    helper.make_tensor_value_info("c1", TensorProto.FLOAT, conv_param_shape),
    helper.make_tensor_value_info("839", TensorProto.INT64,[2]) 
]

graph = helper.make_graph(
    nodes = [conv_node, shape_node,slice_node,concat_node],
    name = "slice_graph",
    inputs = [top_in],
    outputs = [top_out],
    value_info = value_info,
)

### Create full model

In [29]:
model_config = {}
model_config["opset_imports"] = [helper.make_operatorsetid("",9)]
modelProto = helper.make_model(graph,**model_config)
model = ModelWrapper(modelProto)

model.set_initializer("starts",np.array([0]))
model.set_initializer("ends",np.array([2]))
model.set_initializer("axes",np.array([0]))
model.set_initializer("839",np.array([10,10]))

model.transform(InferShapes())
model.save("onnx_model/slice.onnx")

### Finn build

In [40]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

model_file = "slice.onnx"

final_output_dir = "output_files/slice"

#Delete previous run results if exist
if os.path.exists(final_output_dir):
    shutil.rmtree(final_output_dir)
    print("Previous run results deleted!")

cfg = build.DataflowBuildConfig(
    output_dir          = final_output_dir,
    mvau_wwidth_max     = 80,
    target_fps          = 1000000,
    synth_clk_period_ns = 10.0,
    board               = "KV260_SOM",
#     steps               = "estimate_only_dataflow_steps",
    shell_flow_type     = build_cfg.ShellFlowType.VIVADO_ZYNQ,
    generate_outputs=[
        build_cfg.DataflowOutputType.BITFILE,
        build_cfg.DataflowOutputType.PYNQ_DRIVER,
        build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
    ]
)

In [41]:
%%time
build.build_dataflow_cfg(model_file, cfg)

Traceback (most recent call last):
  File "/workspace/finn/src/finn/builder/build_dataflow.py", line 166, in build_dataflow_cfg
    model = transform_step(model, cfg)
  File "/workspace/finn/src/finn/builder/build_dataflow_steps.py", line 327, in step_create_dataflow_partition
    assert len(sdp_nodes) == 1, "Only a single StreamingDataflowPartition supported."
AssertionError: Only a single StreamingDataflowPartition supported.


Building dataflow accelerator from slice.onnx
Intermediate outputs will be generated in /workspace/results
Final outputs will be generated in output_files/slice
Build log is at output_files/slice/build_dataflow.log
Running step: step_qonnx_to_finn [1/17]
Running step: step_tidy_up [2/17]
Running step: step_streamline [3/17]
Running step: step_convert_to_hls [4/17]
Running step: step_create_dataflow_partition [5/17]
> /workspace/finn/src/finn/builder/build_dataflow_steps.py(327)step_create_dataflow_partition()
    325     )
    326     sdp_nodes = parent_model.get_nodes_by_op_type("StreamingDataflowPartition")
--> 327     assert len(sdp_nodes) == 1, "Only a single StreamingDataflowPartition supported."
    328     sdp_node = sdp_nodes[0]
    329     sdp_node = getCustomOp(sdp_node)

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
Build failed
CPU times: user 201 ms, sys: 44.3 ms, total: 246 ms
Wall time: 3.62 s


-1